# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint

## Learning Objectives

At the end of the experiment, you will be able to :

* obtain the mfcc features from audio samples 
* train the classifier with these mfcc features
* classify the audio samples into 'yes' and 'no'

In [ ]:
#@title Experiment Walkthrough Video

from IPython.display import HTML

HTML("""<video width="854" height="480" controls>
<source src="https://cdn.exec.talentsprint.com/content/yes_no_classifier.mp4" type="video/mp4">
</video>""")

## Dataset 

### Description
The dataset used in the experiment contains 2661 audio samples with the extension .wav. 

Each audio sample has utterances 'yes' or 'no'

The naming convention of the sample is of the format 'yes_no/' followed by '10' (representing 'yes') or '11' (representing 'no')

Ex:   

        'yes_no/10_g38_46.wav'    ---> yes audio sample
        'yes_no/11_G6_11.wav'     ---> no audio sample



## Domain Information

When we listen to an audio sample it changes constantly. This means that speech is non-stationary signal. Therefore, normal signal processing techniques cannot be applied to get features from audio. However, if the speech signal is observed using a very small duration window, the speech content in that small duration appears to be  stationary. That brought in the concept of short-time processing of speech. 

MFCC is a techique for short-time processing of speech. 

**Note: While it is a bonus to understand 'how' a speech signal is converted to a 'representation' by MFCC (or other similar speech processing techniques), it is not mandatory to understand the same. It is sufficient to realize that you're generating a unique numeric representation (a set of numbers) of a speech signal and that this unique representation helps you run your machine learning algorithms over the samples.**

## AIML Technique

In this short-time processing technique MFCC, a small duration window (say 25 milli sec) is considered for processing of the audio samples at a time. This small duration is called a frame. Now, for each of the frames, MFCC features are computed which give a compact representation of the audio samples. A spectogram is computed for audio samples which gives a heat map of frequencies vs time from the series of spectral vectors generated from audio samples.

This representation is obtained by keeping in mind that humans are much better at grasping small changes in the audio at low frequencies than audio at high frequencies. So mel-scale converts high frequencies to human graspable frequencies. 

In [ ]:
! wget https://cdn.talentsprint.com/aiml/Experiment_related_data/yes_no.zip
! unzip yes_no.zip

### Importing Required Packages

In [ ]:
import numpy as np
import librosa

# Visualization
import matplotlib.pyplot as plt
import librosa.display

# Import glob to perform pattern matching to find files
import glob

# Import train_test_split to split the data into train and test sets
from sklearn.model_selection import train_test_split

# Import MLPClassifier to be trained on features of wav files
from sklearn.neural_network import MLPClassifier

#### About glob.iglob:

The glob library  provides methods for traversing the file system and returning files that matched a defined set of glob patterns. Here it helps traverse file name matching the ".wav" pattern

**Note:** Refer to  [glob.iglob](https://docs.python.org/3/library/glob.html)

In [ ]:
labels = []
wave_files = []
for filename in glob.iglob('yes_no/*.wav', recursive=True):
  # Store the labels of each wav file in a list
  labels.append(int((filename.split("/")[-1]).split("_")[0])) # Split the file name to get the labels of the audio files
  # Store the wav files in a list
  wave_files.append(filename)

In [ ]:
# Check the length of wav files
len(wave_files)

### Mel Spectrogram

Human ears tend to listen to sounds on the log scale. That means, at lower frequencies we can detect small changes, but at higher frequencies our ears become less sensitive to small changes. For example, the difference between 10 Hz and 20 Hz would sound almost the same to us as that between 1000 Hz and 2000 Hz. 

Let us compute the Mel Spectrogram features using a convenient function in the `librosa` library in Python:

Below is the function for loading the wavefile and extracting the melspectogram features at a sample rate

* Refer to [librosa.load](http://man.hubwiz.com/docset/LibROSA.docset/Contents/Resources/Documents/generated/librosa.core.load.html)

* Refer to [librosa.feature.melspectrogram](https://librosa.org/doc/main/generated/librosa.feature.melspectrogram.html)

In [ ]:
def get_melspectrogram_features(filename):
    y, sr = librosa.load(filename)
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    # The log of the spectrogram is a better parameter to observe rather than the spectrogram itself.
    # Convert to log scale (dB). We'll use the peak power (max) as reference.
    log_S = librosa.power_to_db(S, ref=np.max)
    return log_S,sr

Let us try to extract the features for one of the wave file and plot the melspectrogram 

**Note:** Refer to [librosa.display.specshow](https://librosa.org/doc/latest/generated/librosa.display.specshow.html)


In [ ]:
features, sr = get_melspectrogram_features(wave_files[0])
plt.figure(figsize=(12, 4))
librosa.display.specshow(features, sr=sr, x_axis='time', y_axis='mel')
plt.title('Mel power spectrogram ')
plt.colorbar(format='%+02.0f dB')
plt.tight_layout()

In [ ]:
# Play the audio from wave file
import IPython.display as ipd
ipd.Audio(wave_files[0]) 

### Extract the MFCC features for all Audio files


* Refer to [librosa.load](http://man.hubwiz.com/docset/LibROSA.docset/Contents/Resources/Documents/generated/librosa.core.load.html)


*   Refer to [librosa.feature.mfcc](https://librosa.org/doc/0.7.2/generated/librosa.feature.mfcc.html)  

In [ ]:
n_mfcc = 30  # mfcc features

Since our ear cannot response to very fast change of speech data, we normally cut the speech data into frames before analysis.


In [ ]:
# Extract MFCC features 

def get_mfcc_features(filename, sr=7000):
    frames = 15          # Get the MFCC features with frame size 15  
    # Load the audio files with the sampling rate 7000.
    y, sr = librosa.load(filename, sr=sr)
    features = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=30)
    # We need to split the signal into short-time frames as the frequencies in a signal change over time.
    # So in most cases we don't do the Fourier transform across the entire signal.
    if features.shape[1] < frames :
        features = np.hstack((features, np.zeros((n_mfcc, frames - features.shape[1]))))
    elif features.shape[1] > frames:
        features = features[:, :frames]
    return features

### Extracting mfcc features for all the samples

In [ ]:
features = []                  # It takes some time to run this cell
for filename in wave_files:
    fea = get_mfcc_features(filename)
    features.append(fea)

In [ ]:
# Check the shape of the features
features[0].shape

### Split the mfcc features into train and test sets with 80-20 split 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.2)

Get the length of the train and test data

In [ ]:
len(X_train), len(X_test), len(y_train), len(y_test)

In [ ]:
# Convert the X_train to an array and reshape from 2128*30*15 to 2128*450

X_train = np.array(X_train)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1] *  X_train.shape[2])

# Convert the X_test to an array and reshape from 533*30*15 to 533*450

X_test = np.array(X_test)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1] *  X_test.shape[2])

In [ ]:
# Get the shape of X_train, X_test
X_train.shape, X_test.shape

### Apply MLP classifier on the MFCC features

**Note:** Refer to [MLP Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) from sklearn

In [ ]:
# Create a object for MLPClassifier 
clf = MLPClassifier(activation='logistic', hidden_layer_sizes=(100,50), random_state=1, solver='adam')

# Fit the train fetaures and labels to the classifier
clf.fit(np.array(X_train), np.array(y_train))  

# Predict the labels of the test features
predicted_values = clf.predict(X_test) 

In [ ]:
# Get the accuracy score by comparing predicted labels and test labels
from sklearn.metrics import accuracy_score
accuracy_score(y_test,predicted_values)